In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
#Reading the csv file into a dataframe
data = pd.read_csv('sonar.all-data.csv', header=None)

In [4]:
# View the top five rows of the data
data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [5]:
X = data.drop(60, axis=1)

In [6]:
print(X.shape)
X.head()

(208, 60)


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [7]:
Y = data[60]
print(Y.head())
Y.replace({'R':0, 'M':1}, inplace=True)

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object


In [8]:
Y

0      0
1      0
2      0
3      0
4      0
      ..
203    1
204    1
205    1
206    1
207    1
Name: 60, Length: 208, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_trn, X_tst, y_trn, y_tst = train_test_split(X, Y, test_size=0.2, random_state=0)

In [10]:
def sigmoid(x):
    '''
    Returns the sigmoid of x
    '''
    return 1/(1+np.exp(-x))

In [11]:
def init_param(X):
    '''
    Args: X -> input matrix
    Returns: a tuple of initialised parameters , w vector and b for the bias
    '''
    np.random.seed(0)
    w = np.random.rand(X.shape[1])
    b = 0
    param = {'w': w,
             'b': b
            }
    return param


In [12]:
def Cost_func(X, Y, param):
    '''
    Computes the cost and gradient value for a given X, Y, param
    Args:
        X => input feautre matrix 
        Y => output array
        param => parameters
    Returns: 
        J => Cost 
        grad => gradient values for w and b
    '''
    m = len(X)
    prob = np.dot(param['w'], X.T) + param['b']
    pred = sigmoid(prob)
    
    J = - np.sum((Y* np.log(pred)) + ((1-Y)*(np.log(1-pred))))/m
    
    dw = (1/m)* np.dot((pred - Y), X)
    db = (1/m)* np.sum(pred -Y)
    
    grad= {'dw': dw,
           'db': db}
    
    return J, grad

In [13]:
def optimize(X, Y, param, num_iter, learning_rate):
    '''
    Optimize the paramenter using gradient descent.
    Args:
        X => input feature matrix
        Y => output array
        param => parameters which are updated
        num_iter => total number of itertions
        learning_rate => learning rate to specify the step size during gradient descent
    Returns:
        param => updated parameters' array
        cost => final cost        
    '''
    for i in range(num_iter):
        cost, grad = Cost_func(X, Y, param)
        param['w'] = param['w']-(learning_rate*grad['dw'])
        param['b'] = param['b']-(learning_rate*grad['db'])
        
        
    return param, cost

In [14]:
def predict(X, param):
    '''
    predicts the target value, Y_pred
    Args: X => input feature matrix,
          param => parameter vector (w, b)
    Returns: Array of predictions
    '''
    Y_pred=[]
    prob= np.dot(param['w'], X.T) + param['b']
    pred = sigmoid(prob)
    for i in range(len(X)):
        if pred[i] > 0.5:
            Y_pred.append(1)
        else:
            Y_pred.append(0)
            
    return np.array(Y_pred).T
        

In [15]:
def accuracy(Y, Y_pred):
    '''
    Returns the accuracy for the predictions
    '''
    return np.sum(Y == Y_pred) / len(Y)

In [16]:
def main(X, Y, learning_rate=0.01, num_iter=5000):
    '''
    Combining all the possible 
    '''
    param = init_param(X)
    cost, grad= Cost_func(X, Y, param)
    param, cost_f = optimize(X, Y, param, num_iter, learning_rate)
    
    y_pred = predict(X, param)
    acc = accuracy(Y, y_pred)
    
    print('Final_cost \t', cost_f)
    print('Accuracy_train \t', acc)
    return param, y_pred

In [17]:
param_f = main(X_trn, y_trn)[0]

Final_cost 	 0.47745029246608806
Accuracy_train 	 0.8012048192771084


In [18]:
pred_y = predict(X_tst, param_f)

In [19]:
accuracy(y_tst, pred_y)

0.7619047619047619